# Intro: Demo

[Acknowledgments Page](https://ds100.org/sp24/acks/)

## Software Packages 
We will be using a wide range of different Python software packages.  To install and manage these packages we will be using the Conda environment manager.  The following is a list of packages we will routinely use in lectures and homeworks:

In [ ]:
# Linear algebra, probability
import numpy as np

# Data manipulation
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive visualization library
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

We will learn how to use all of the technologies used in this demo.

For now, just sit back and think critically about the data and our guided analysis.

# 1. Starting with a Question: **Who are the students of Data 100?**

<img src="images/ask.png" width="300px" />

This is a pretty vague question but let's start with the goal of learning something about the students in the class.

Here are some "simple" questions:
1. How many students do we have?
1. What are your majors?
1. What year are you?
1. How did your major enrollment trend change over time?


# 2. Data Acquisition and Cleaning 

**In Data 100 we will study various methods to collect data.**

<img src="images/data_acquisition.PNG" width="300px" />

To answer this question, I downloaded the course roster and extracted everyone's names and majors.

In [ ]:
# pd stands for pandas, which we will learn starting from next lecture
# Some pandas syntax shared with data8's datascience package
majors = pd.read_csv("data/majors-sp24.csv")
names = pd.read_csv("data/names-sp24.csv")

# 3. Exploratory Data Analysis

**In Data 100 we will study exploratory data analysis and practice analyzing new datasets.**

<img src="images/understand_data.PNG" width="300px" />

I didn't tell you the details of the data! Let's check out the data and infer its structure. Then we can start answering the simple questions we posed.

### Peeking at the Data

In [ ]:
# Let's peek at the first 20 rows of the majors dataframe

majors.head(20)

In [ ]:
# Let's peek at the first 5 rows (default) of the names dataframe

names.head()

### What is one potential issue we may need to address in this data?

**Answer:**
Some names appear capitalized. 

In the above sample we notice that some of the names are capitalized and some are not.  This will be an issue in our later analysis so let's convert all names to lower case.

In [ ]:
names['Name'] = names['Name'].str.lower()

In [ ]:
names.head()

### Exploratory Data Analysis on `names` dataset

#### What is the most common first letter in names? What is its distribution? 


In [ ]:
# Below are the most common, in descending frequency

first_letter = names['Name'].str[0].value_counts()
first_letter.head()

In [ ]:
# Let's visualize this first letter distribution 

plt.bar(first_letter.index, first_letter.values)
plt.xlabel('First Letter')
plt.ylabel('Frequency')
plt.title('First Letter Frequency Distribution')
plt.show()


In the United States, "J" and "A" names are the most popular first initials. Seems like our visualization also reflects this!

#### What is the distribution of the length of names? 


In [ ]:
name_lengths = names['Name'].str.len()

plt.hist(name_lengths, bins=range(min(name_lengths), max(name_lengths) + 2), edgecolor='black')
plt.xlabel('Name Length')
plt.ylabel('Frequency')
plt.title('Distribution of Length of Names')
average_length = name_lengths.sum() / len(name_lengths)
plt.axvline(average_length, color='red', linestyle='dashed', linewidth=1, label=f'Average: {average_length:.2f}')
plt.legend()
plt.xticks(range(min(name_lengths), max(name_lengths) + 1))
plt.show()

The average length of names in the United States is also around 6 letters!

#### How many records do we have?

In [ ]:
print(len(names))
print(len(majors))

Based on what we know of our class, each record is most likely a student.

#### Understanding the structure of data

It is important that we understand the meaning of each field and how the data is organized.

In [ ]:
names.head()

It appears that one student has an erroneous role given as "#REF!". What else can we learn about this student? Let's see their name.

In [ ]:
# Boolean index to find rows where Role is #REF!
names[names['Name'] == "#ref!"]

Though this single bad record won't have much of an impact on our analysis, we can clean our data by removing this record.

In [ ]:
names = names[names['Name'] != "#ref!"]

**Double check**: Let's double check that our record removal only removed the single bad record.

In [ ]:
names['Name'].value_counts().to_frame()  # Again, counts of unique Roles

Remember we loaded in two files. Let's explore the fields of `majors` and check for bad records:

### Exploratory Data Analysis on `majors` dataset

In [ ]:
majors.columns   # Get column names

In [ ]:
majors['Terms in Attendance'].value_counts().to_frame()

It looks like numbers represent semesters, `G` represents graduate students. But we do still have a bad record:

In [ ]:
majors[majors['Terms in Attendance'] == "#REF!"]

In [ ]:
majors = majors[majors['Terms in Attendance'] != "#REF!"]
majors['Terms in Attendance'].value_counts().to_frame()

Detail: The deleted `majors` record number is different from the record number of the bad `names` record. So while the number of records in each table matches, the row indices don't match, so we'll have to keep these tables separate in order to do our analysis.

### Summarizing the Data

We will often want to numerically or visually summarize the data. The `describe()` method provides a brief high level description of our data frame. 

In [ ]:
names.describe()

**Q: What do you think `top` and `freq` represent?**

**Answer:** `top`: most frequent entry, `freq`: the frequency of that entry

In [ ]:
majors.describe()

---
# 4. Analysis: Understanding the World

<img src="images/understand_world.PNG" width="300px" />


### What are your majors?

What are the top majors:

In [ ]:
majors_count = (       # Method chaining in pandas
    majors['Majors']
    .value_counts()
    .sort_values(ascending=False) # Highest first
    .to_frame()
    .head(20)          # Get the top 20
)

majors_count

### We will often use visualizations to make sense of data
**In Data 100, we will deal with many different kinds of data (not just numbers) and we will study techniques to describe types of data.**

**How can we summarize the Majors field?** A good starting point might be to use a bar plot:

In [ ]:
# Interactive using plotly

fig = px.bar(majors_count.loc[::-1], orientation='h')
fig.update_layout(showlegend=False, 
                  xaxis_title='Count', 
                  yaxis_title='Major',
                  autosize=False, 
                  width=800, 
                  height=500)

### What year are you?

In [ ]:
fig = px.histogram(majors['Terms in Attendance'].sort_values(),
                   histnorm='probability')

fig.update_layout(showlegend=False,
                  xaxis_title="Term",
                  yaxis_title="Fraction of Class",
                  autosize=False, 
                  width=800, 
                  height=250)

In [ ]:
# Replacing terms in attendance data with the degree objective
majors.loc[majors.loc[:, 'Terms in Attendance'] != 'G', 'Terms in Attendance'] = 'Undergraduate'
majors.loc[majors.loc[:, 'Terms in Attendance'] == 'G', 'Terms in Attendance'] = 'Graduate'


majors.rename(columns={'Terms in Attendance': 'Ungrad Grad'}, inplace=True)

majors.describe()

# 1. New Questions

- **What is the ratio between graduate and undergraduate students in Data 100, and how does it compare with campus distribution?**

- **What is the proportion of different majors in Data 100, and how does it compare with historical campus trends?**

<img src="images/ask.png" width="300px" />

We often ask this question because we want to improve the data science program here in Berkeley, especially since it has now grown into a new college—[College of Computing, Data Science, and Society](https://data.berkeley.edu/)—Berkeley's first new college in 50 years.



## How could we answer this question?



In [ ]:
print(majors.columns)
print(names.columns)


---

## We don't have the data.

Where can we get the data?

<img src="images/data_acquisition.PNG" width="300px" />


---

# UC Berkeley Student Headcount by Major and Demographics


[Data](https://pages.github.berkeley.edu/OPA/our-berkeley/student-headcount-by-major.html) comes from UC Berkeley's data warehouse reporting system, Cal Answers. Headcount figures are based on the Fall and Spring census snapshots, which occur at the end of the 5th week of the term.

# 2. Acquire data programmatically

Note 1: In the following, we download the data programmatically to ensure that the process is reproducible.

Note 2: We also load the data directly into Python.

**In Data 100 we will think a bit more about how we can be efficient in our data analysis to support processing large datasets.**

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1J7tz3GQLs3M6hFseJCE9KhjVhe4vKga8Q2ezu0oG5sQ/gviz/tq?tqx=out:csv"

university_majors = pd.read_csv(url, 
                                usecols = ['Academic Yr', 'Semester', 'Ungrad Grad', 
                                           'Entry Status', 'Major', 'Student Headcount'])

# 3. Exploratory Data Analysis on Campus Data

<img src="images/understand_data.PNG" width="300px" />


In [ ]:
# Examining the data
university_majors

The data is reported on a semester basis. We will aggregate data across different semesters in a year by taking average of Fall and Spring semester enrollment information.

In [ ]:
# Reporting student data based on academic year
university_majors = (university_majors.groupby(
    ['Academic Yr', 'Ungrad Grad', 'Entry Status', 'Major'], as_index = False)[["Student Headcount"]]
                     .mean()
                    )

university_majors

## What is the historical distribution of graduate and undergraduate students at Berkeley? 

In [ ]:
university_grad_vs_ungrd = (university_majors.groupby(
    ['Academic Yr', 'Ungrad Grad'], as_index = False)[["Student Headcount"]]
                            .sum()
                           )

proportions = university_grad_vs_ungrd.pivot(index='Academic Yr', columns='Ungrad Grad', values='Student Headcount')
proportions['Total'] = proportions['Undergraduate'] + proportions['Graduate']
proportions['Undergrad Proportion'] = proportions['Undergraduate'] / proportions['Total']
proportions['Grad Proportion'] = proportions['Graduate'] / proportions['Total']


fig = px.bar(proportions.reset_index(), 
             x='Academic Yr', 
             y=['Undergraduate', 'Graduate'],
             title='Number of Grad vs. Undergrad Students',
             labels={'value': 'Number of Students'},
             color_discrete_map={'Undergraduate': 'blue', 'Graduate': 'orange'})

fig.update_layout(barmode='relative', autosize=False, width=800, height=600)
fig.show()


# 4.1. Ratio between graduate and undergraduate students in Data 100, and its comparison with campus distribution

In [ ]:
data100_grad = majors['Ungrad Grad'].loc[majors['Ungrad Grad'] == 'Graduate'].count()

data100_undergrad = majors['Ungrad Grad'].loc[majors['Ungrad Grad'] == 'Undergraduate'].count()

print("Number of graduate students in Data 100: ", data100_grad)
print("Number of undergraduate students in Data 100: ", data100_undergrad)

In [ ]:
data100_row = {'Graduate':[data100_grad], 
               'Undergraduate':[data100_undergrad], 
               'Total':[data100_grad + data100_undergrad], 
               'Undergrad Proportion':[data100_undergrad / (data100_grad + data100_undergrad)],
               'Grad Proportion':[data100_grad / (data100_grad + data100_undergrad)], 
               }

new_row_df = pd.DataFrame(data100_row)

proportions.loc['Data 100'] = new_row_df.iloc[0]


fig = px.bar(proportions.reset_index(), 
             x='Academic Yr', 
             y=['Undergrad Proportion', 'Grad Proportion'],
             title='Proportions of Grad vs. Undergrad Students',
             labels={'value': 'Proportion'},
             color_discrete_map={'Undergrad Proportion': 'blue', 'Grad Proportion': 'orange'})


fig.update_layout(barmode='relative', autosize=False, width=800, height=600)
fig.show()

# 4.2. Proportion of different majors in Data 100, and their historical emrollment trends

In [ ]:
data100_top_20_majors = (       # Method chaining in pandas
    majors['Majors']
    .value_counts()
    .sort_values(ascending=False) # Highest first
    .to_frame()
    .head(20)          # Get the top 20
)

major_trends = university_majors.groupby(['Academic Yr', 'Major'], 
                                         as_index = False)[["Student Headcount"]].sum()


print("Top 20 majors at Berkeley in 2022-23")
major_trends[major_trends.loc[:, 'Academic Yr'] == '2022-23'].sort_values('Student Headcount', ascending=False).head(20)

In [ ]:
print("Top 20 majors at Berkeley since 2013")
major_trends.groupby(['Major'], as_index = False)[['Student Headcount']].sum().sort_values('Student Headcount', ascending=False).head(20)
     

In [ ]:
print("Top 20 majors at Berkeley in Data 100")
print(data100_top_20_majors)

In [ ]:
data100_top_20_majors.index = data100_top_20_majors.index.str.rsplit(' ', n=1).str[0]
print("Top 20 majors at Berkeley in Data 100")
print(data100_top_20_majors)

In [ ]:
fig = px.line(major_trends[major_trends["Major"].isin(data100_top_20_majors.index)], 
                        x = "Academic Yr", y = "Student Headcount", color = "Major")

fig.update_layout(autosize=False, width=800, height=600)
fig.show()    

In [ ]:
data100_top_19_majors = data100_top_20_majors.iloc[1:,:]

fig = px.line(major_trends[major_trends["Major"].isin(data100_top_19_majors.index)], 
                        x = "Academic Yr", y = "Student Headcount", color = "Major")

fig.update_layout(autosize=False, width=800, height=600)
fig.show()  